In [1]:
# do not touch this cell 
%load_ext jupyter_record


# Imports


In [3]:
%%git_commit

import pandas as pd



# Dataset Load




In [10]:
%%git_commit

training = pd.read_csv('../archive/Combined_Flights_2018.csv').sample(1000000)
testing = pd.read_csv('../archive/Combined_Flights_2019.csv').sample(1000000)


In [8]:
%%git_commit

training = training.dropna()
testing = testing.dropna()





In [9]:
%%git_commit

dropped_column = ['Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline', 'DepTime', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'DepDelayMinutes', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'DistanceGroup', 'DivAirportLandings']




In [11]:
%%git_commit

for column in dropped_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)


In [13]:
%%git_commit


training


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,ArrDelayMinutes,...,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,CRSArrTime
528408,2018-10-11,American Airlines Inc.,EWR,CLT,False,True,1025,1022.0,0.0,NaN,...,11057,1105703,31057,"Charlotte, NC",NC,37,North Carolina,36,0.0,1225
3568735,2018-05-23,Southwest Airlines Co.,BWI,RDU,False,False,1555,1608.0,13.0,11.0,...,14492,1449202,34492,"Raleigh/Durham, NC",NC,37,North Carolina,36,0.0,1700
4540346,2018-07-19,Frontier Airlines Inc.,DEN,DCA,False,False,1630,1629.0,0.0,0.0,...,11278,1127805,30852,"Washington, DC",VA,51,Virginia,38,0.0,2158
4318100,2018-07-23,Southwest Airlines Co.,LBB,DAL,True,False,845,NaN,NaN,NaN,...,11259,1125903,30194,"Dallas, TX",TX,48,Texas,74,NaN,955
4737105,2018-08-21,Mesa Airlines Inc.,OKC,DFW,False,False,2019,2007.0,0.0,0.0,...,11298,1129806,30194,"Dallas/Fort Worth, TX",TX,48,Texas,74,0.0,2121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2878820,2018-03-16,Southwest Airlines Co.,MKE,CLE,False,False,1030,1126.0,56.0,43.0,...,11042,1104205,30647,"Cleveland, OH",OH,39,Ohio,44,1.0,1240
3919214,2018-06-22,United Air Lines Inc.,ORD,SMF,False,False,1030,1028.0,0.0,48.0,...,14893,1489302,33192,"Sacramento, CA",CA,6,California,91,0.0,1256
2804463,2018-03-10,Southwest Airlines Co.,SJU,MCO,False,False,730,733.0,3.0,0.0,...,13204,1320402,31454,"Orlando, FL",FL,12,Florida,33,0.0,950
5684979,2018-09-17,United Air Lines Inc.,MSY,SFO,False,False,630,641.0,11.0,0.0,...,14771,1477104,32457,"San Francisco, CA",CA,6,California,91,0.0,904


# Time Series Data 


In [ ]:
%%git_commit




# Model Training



In [ ]:
%%git_commit


# Present to Students


In [ ]:
%%git_commit
